# Imports

In [1]:
import random
import numpy as np
import keras.models as Km
import keras.layers as Kl
import tensorflow as tf

* Make a move for X -- pick best move, train the model based on picked state and associated reward, update state
* determine best move -- based on Q grid
* Train Model -- x value is the state as an array, y value (target) is the q-value of best next move + reward at current state 
* Calc_value_of_state -- use model to predict value of state
* calc target -- is the q-value of best next move + reward at current state 
* run experiment -- runs through a tictactoe game 

# Piece Class

In [2]:
from enum import Enum

class Piece(Enum):
    EMPTY = 'EMPTY'
    BLACK = 'BLACK'
    WHITE = 'WHITE'
    RED = ' RED '

# Board Class

In [3]:
import copy

class Board:
    def __init__(self):
        self.pieces = [[[Piece.EMPTY for k in range(3)] for j in range(3)] for i in range(3)]
        self.winningRuns = self.getWinningRuns()

    def validMove(self,x,y,z,dir):
        if not x in range(3) or not y in range(3) or not z in range(3):
              return False
        if dir == 'UP':
              return (z == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y][z-1] == Piece.EMPTY or self.pieces[x][y][z-2] == Piece.EMPTY)
        if dir == 'DOWN':
              return (z == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y][z+1] == Piece.EMPTY or self.pieces[x][y][z+2] == Piece.EMPTY)
        if dir == 'LEFT':
              return (x == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x-1][y][z] == Piece.EMPTY or self.pieces[x-2][y][z] == Piece.EMPTY)
        if dir == 'RIGHT':
              return (x == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x+1][y][z] == Piece.EMPTY or self.pieces[x+2][y][z] == Piece.EMPTY)
        if dir == 'FRONT':
              return (y == 2) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y-1][z] == Piece.EMPTY or self.pieces[x][y-2][z] == Piece.EMPTY)
        if dir == 'BACK':
              return (y == 0) and (self.pieces[x][y][z] == Piece.EMPTY or self.pieces[x][y+1][z] == Piece.EMPTY or self.pieces[x][y+2][z] == Piece.EMPTY)
        else:
              return False
            
    def move(self,x,y,z,dir,player: Piece):
        if not self.validMove(x,y,z,dir):
             raise ValueError
        else:
            if (self.pieces[x][y][z] == Piece.EMPTY):
                self.pieces[x][y][z] = player
            else:
                if dir == 'UP':
                    if (self.pieces[x][y][z-1] == Piece.EMPTY):
                        self.pieces[x][y][z-1] = self.pieces[x][y][z]
                        self.pieces[x][y][z] = player
                    else:
                        self.pieces[x][y][z-2] = self.pieces[x][y][z-1]
                        self.pieces[x][y][z-1] = self.pieces[x][y][z]
                        self.pieces[x][y][z] = player
                elif dir == 'DOWN':
                      if (self.pieces[x][y][z+1] == Piece.EMPTY):
                          self.pieces[x][y][z+1] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                      else:
                          self.pieces[x][y][z+2] = self.pieces[x][y][z+1]
                          self.pieces[x][y][z+1] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                elif dir == 'LEFT':
                      if (self.pieces[x-1][y][z] == Piece.EMPTY):
                          self.pieces[x-1][y][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                      else:
                          self.pieces[x-2][y][z] = self.pieces[x-1][y][z]
                          self.pieces[x-1][y][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                elif dir == 'RIGHT':
                      if (self.pieces[x+1][y][z] == Piece.EMPTY):
                          self.pieces[x+1][y][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                      else:
                          self.pieces[x+2][y][z] = self.pieces[x+1][y][z]
                          self.pieces[x+1][y][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                elif dir == 'FRONT':
                      if (self.pieces[x][y-1][z] == Piece.EMPTY):
                          self.pieces[x][y-1][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                      else:
                          self.pieces[x][y-2][z] = self.pieces[x][y-1][z]
                          self.pieces[x][y-1][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                elif dir == 'BACK':
                      if (self.pieces[x][y+1][z] == Piece.EMPTY):
                          self.pieces[x][y+1][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player
                      else:
                          self.pieces[x][y+2][z] = self.pieces[x][y+1][z]
                          self.pieces[x][y+1][z] = self.pieces[x][y][z]
                          self.pieces[x][y][z] = player

    def getGameState(self):
        gameState = "+----------------------\n"
        gameState += "| \ " + self.pieces[0][2][0].value + "  " + self.pieces[1][2][0].value + "  " + self.pieces[2][2][0].value + " \\\n"
        gameState += "|   \                     \\\n"
        gameState += "|     \ " + self.pieces[0][1][0].value + "  " + self.pieces[1][1][0].value + "  " + self.pieces[2][1][0].value + " \\\n"
        gameState += "|       \                     \\\n"
        gameState += "|         \ " + self.pieces[0][0][0].value + "  " + self.pieces[1][0][0].value + "  " + self.pieces[2][0][0].value + " \\\n"
        gameState += "|          ---------------------|\n"
        gameState += "|   " + self.pieces[0][2][1].value + " |" + self.pieces[1][2][1].value + "  " + self.pieces[2][2][1].value + "         |\n"
        gameState += "|         |                     |\n"
        gameState += "|       " + self.pieces[0][1][1].value + "  " + self.pieces[1][1][1].value + "  " + self.pieces[2][1][1].value + "     |\n"
        gameState += "|         |                     |\n"
        gameState += "|         | " + self.pieces[0][0][1].value + "  " + self.pieces[1][0][1].value + "  " + self.pieces[2][0][1].value + " |\n"
        gameState += "|         |                     |\n"
        gameState += " \ " + self.pieces[0][2][2].value + "  " + self.pieces[1][2][2].value + "  " + self.pieces[2][2][2].value + "          |\n"
        gameState += "   \      |                     |\n"
        gameState += "     \ " + self.pieces[0][1][2].value + "  " + self.pieces[1][1][2].value + "  " + self.pieces[2][1][2].value + "      |\n"
        gameState += "       \  |                     |\n"
        gameState += "         \| " + self.pieces[0][0][2].value + "  " + self.pieces[1][0][2].value + "  " + self.pieces[2][0][2].value + " |\n"
        gameState += "           ---------------------+\n\n"
        return gameState
    

    def getWinningRuns(self):
        runs = []

        runs.append([(0,0,0),(0,0,1),(0,0,2)])
        runs.append([(0,0,0),(0,1,0),(0,2,0)])
        runs.append([(0,0,0),(1,0,0),(2,0,0)])

        runs.append([(2,2,0),(1,2,0),(0,2,0)])
        runs.append([(2,2,0),(2,1,0),(2,0,0)])
        runs.append([(2,2,0),(2,2,1),(2,2,2)])

        runs.append([(0,2,2),(0,1,2),(0,0,2)])
        runs.append([(0,2,2),(1,2,2),(2,2,2)])
        runs.append([(0,2,2),(0,2,1),(0,2,0)])

        runs.append([(2,0,2),(2,0,1),(2,0,0)])
        runs.append([(2,0,2),(1,0,2),(0,0,2)])
        runs.append([(2,0,2),(2,1,2),(2,2,2)])
        # Front
        runs.append([(0,0,0),(1,0,1),(2,0,2)])
        runs.append([(0,0,2),(1,0,1),(2,0,0)])
        runs.append([(1,0,0),(1,0,1),(1,0,2)])
        runs.append([(0,0,1),(1,0,1),(2,0,1)])
        # Top
        runs.append([(0,0,0),(1,1,0),(2,2,0)])
        runs.append([(0,2,0),(1,1,0),(2,0,0)])
        runs.append([(0,1,0),(1,1,0),(2,1,0)])
        runs.append([(1,2,0),(1,1,0),(1,0,0)])
        # Left
        runs.append([(0,0,0),(0,1,1),(0,2,2)])
        runs.append([(0,0,2),(0,1,1),(0,2,0)])
        runs.append([(0,0,1),(0,1,1),(0,2,1)])
        runs.append([(0,1,0),(0,1,1),(0,1,2)])
        # Back
        runs.append([(0,2,2),(1,2,1),(2,2,0)])
        runs.append([(0,2,0),(1,2,1),(2,2,2)])
        runs.append([(1,2,0),(1,2,1),(1,2,2)])
        runs.append([(0,2,1),(1,2,1),(2,2,1)])
        # Right
        runs.append([(2,0,2),(2,1,1),(2,2,0)])
        runs.append([(2,0,0),(2,1,1),(2,2,2)])
        runs.append([(2,0,1),(2,1,1),(2,2,1)])
        runs.append([(2,1,0),(2,1,1),(2,1,2)])
        # Bottom
        runs.append([(2,0,2),(1,1,2),(0,2,2)])
        runs.append([(0,0,2),(1,1,2),(2,2,2)])
        runs.append([(0,1,2),(1,1,2),(2,1,2)])
        runs.append([(1,0,2),(1,1,2),(1,2,2)])
        # Corners
        runs.append([(0,0,0),(1,1,1),(2,2,2)])
        runs.append([(2,0,0),(1,1,1),(0,2,2)])
        runs.append([(2,2,0),(1,1,1),(0,0,2)])
        runs.append([(0,2,0),(1,1,1),(2,0,2)])
        # Edges
        runs.append([(1,0,0),(1,1,1),(1,2,2)])
        runs.append([(2,1,0),(1,1,1),(0,1,2)])
        runs.append([(1,2,0),(1,1,1),(1,0,2)])
        runs.append([(0,1,0),(1,1,1),(2,1,2)])
        runs.append([(0,0,1),(1,1,1),(2,2,1)])
        runs.append([(2,0,1),(1,1,1),(0,2,1)])
        # Middles
        runs.append([(1,1,0),(1,1,1),(1,1,2)])
        runs.append([(1,0,1),(1,1,1),(1,2,1)])
        runs.append([(0,1,1),(1,1,1),(2,1,1)])

        return runs

    def getPossibleMoves(self):
        directions = ['UP','DOWN','LEFT','RIGHT','FRONT','BACK']
        moves = []
        for x in range(2):
          for y in range(2):
            for z in range(2):
              for dir in directions:
                if self.validMove(x,y,z,dir):
                  moves.append((x,y,z,dir))
        return moves

    def getWinInOne(self,player: Piece):
        for (x,y,z,dir) in self.getPossibleMoves():
          c = copy.deepcopy(self)
          c.move(x,y,z,dir,player)
          if c.hasWon(player):
            return (x,y,z,dir)
        return None

    def otherPlayer(self,player: Piece):
        return Piece.RED if player == Piece.WHITE else Piece.WHITE

    def getDefendingMove(self,player: Piece):
        potential_moves = []
        for (x,y,z,dir) in self.getPossibleMoves():
          c = copy.deepcopy(self)
          c.move(x,y,z,dir,player)
          if c.getWinInOne(self.otherPlayer(player)) == None:
            potential_moves.append((x,y,z,dir))
        if potential_moves:
          return random.choice(potential_moves)
        return None

    def getWinInTwo(self,player: Piece):
        potential_moves = []
        for (x,y,z,dir) in self.getPossibleMoves():
          c = copy.deepcopy(self)
          c.move(x,y,z,dir,player)
          if c.getWinInOne(self.otherPlayer(player)) == None:
            winner = True
            for (x2,y2,z2,dir2) in c.getPossibleMoves():
              c2 = copy.deepcopy(c)
              c2.move(x2,y2,z2,dir2,self.otherPlayer(player))
              if c2.getWinInOne(player) == None:
                winner = False
            if winner:
              potential_moves.append((x,y,z,dir))
        if potential_moves:
          return random.choice(potential_moves)
        return None

    def getRandomMove(self,player: Piece):
        directions = ['UP','DOWN','LEFT','RIGHT','FRONT','BACK']
        x = random.randint(0,2)
        y = random.randint(0,2)
        z = random.randint(0,2)
        dir = random.choice(directions)
        while not self.validMove(x,y,z,dir):
            x = random.randint(0,2)
            y = random.randint(0,2)
            z = random.randint(0,2)
            dir = random.choice(directions)
        return (x,y,z,dir)

    def hasWon(self,player: Piece):
        for run in self.winningRuns:
            if all(self.pieces[x][y][z] == player for (x,y,z) in run):
                return True
        return False
    
    def gameOver(self):
        return self.hasWon(Piece.RED) or self.hasWon(Piece.WHITE)

# Random Agent

In [4]:
class RandomAgent:
    def __init__(self,player):
        self.player = player

    def getMove(self, board: Board):
        return board.getRandomMove(self.player)

# Easy Agent

In [5]:
class EasyAgent:
    def __init__(self,player):
        self.player = player

    def getMove(self, board: Board):
        winningMove = board.getWinInOne(self.player)
        if winningMove:
          return winningMove
        else:
          return board.getRandomMove(self.player)

# Medium Agent

In [6]:
class MediumAgent:
    def __init__(self,player):
        self.player = player

    def getMove(self, board: Board):
        winningMove = board.getWinInOne(self.player)
        if winningMove:
          return winningMove
        else:
          defendingMove = board.getDefendingMove(self.player)
          if defendingMove:
            return defendingMove
          else:
            return board.getRandomMove(self.player)

# Hard Agent

In [7]:
class HardAgent:
    def __init__(self,player):
        self.player = player

    def getMove(self, board: Board):
        winningMove = board.getWinInOne(self.player)
        if winningMove:
          return winningMove
        else:
          winInTwo = board.getWinInTwo(self.player)
          if winInTwo:
            return winInTwo
          else:
            defendingMove = board.getDefendingMove(self.player)
            if defendingMove:
              return defendingMove
            else:
              return board.getRandomMove(self.player)

# Deep Agent

In [18]:
class DeepAgent:
    def __init__(self,player):
        self.player = player
        model_values_path  = '/content/model_values_new.h5'
        self.model = Km.load_model(model_values_path)

    def getMove(self, board):
        state = board.pieces
        moves = self.getPossibleMoves(state)

        temp_state_list = []
        v = -float('Inf')

        temp_state_values = []
        for x,y,z,direction in moves:
            temp_state = copy.deepcopy(state)
            temp_state = self.move(temp_state,x,y,z,direction)
            temp_state_value =self.calc_value(temp_state)
            temp_state_values.append(temp_state_value)
        
        optimal_move_index = np.argmax(temp_state_values)
        # print(moves[optimal_move_index])
        # print(moves)
        # print(temp_state_values)
        # print(len(moves))
        # print(len(temp_state_values))
        x,y,z,direction = moves[optimal_move_index]

        # new_state = self.move(state,x,y,z,direction,self.player)
        return (x,y,z,direction)

    def move(self,state,x,y,z,direction):
        new_state = copy.deepcopy(state)

        if not self.validMove(state,x,y,z,direction):
              raise ValueError
        else:
            if (state[x][y][z] == Piece.EMPTY):
                new_state[x][y][z] = self.player
            else:
                if direction == 'UP':
                    if (state[x][y][z-1] == Piece.EMPTY):
                        new_state[x][y][z-1] = state[x][y][z]
                        new_state[x][y][z] = self.player
                    else:
                        new_state[x][y][z-2] = state[x][y][z-1]
                        new_state[x][y][z-1] = state[x][y][z]
                        new_state[x][y][z] = self.player
                elif direction ==  'DOWN':
                    if (state[x][y][z+1] == Piece.EMPTY):
                        new_state[x][y][z+1] = state[x][y][z]
                        new_state[x][y][z] = self.player
                    else:
                        new_state[x][y][z+2] = state[x][y][z+1]
                        new_state[x][y][z+1] = state[x][y][z]

                        new_state[x][y][z] = self.player
                elif direction == "LEFT":
                    if (state[x-1][y][z] == Piece.EMPTY):
                        new_state[x-1][y][z] = state[x][y][z]
                        new_state[x][y][z] =self.player
                    else:
                        new_state[x-2][y][z] = state[x-1][y][z]
                        new_state[x-1][y][z] = state[x][y][z]
                        new_state[x][y][z] =self.player
                elif direction == 'RIGHT':
                    if (state[x+1][y][z] == Piece.EMPTY):
                        new_state[x+1][y][z] = state[x][y][z]
                        new_state[x][y][z] = self.player
                    else:
                        new_state[x+2][y][z] = state[x+1][y][z]
                        new_state[x+1][y][z] = state[x][y][z]
                        new_state[x][y][z] = self.player
                elif direction == 'FRONT':
                    if (state[x][y-1][z] == Piece.EMPTY):
                        new_state[x][y-1][z] = state[x][y][z]
                        new_state[x][y][z] =self.player
                    else:
                        new_state[x][y-2][z] = state[x][y-1][z]
                        new_state[x][y-1][z] = state[x][y][z]
                        new_state[x][y][z] = self.player
                elif direction == 'BACK':
                    if (state[x][y+1][z] == Piece.EMPTY):
                        new_state[x][y+1][z] = state[x][y][z]
                        new_state[x][y][z] = self.player
                    else:
                        new_state[x][y+2][z] = state[x][y+1][z]
                        new_state[x][y+1][z] = state[x][y][z]
                        new_state[x][y][z] = self.player
        return new_state

    def getPossibleMoves(self, state):
        moves = []
        for x in range(0,3):
            for y in range(0,3):
                for z in range(0,3):
                    for direction in directions:
                        if self.validMove(state,x,y,z,direction):
                            moves.append((x,y,z,direction))
        return moves

    def validMove(self,state,x,y,z,direction):
        if not x in range(3) or not y in range(3) or not z in range(3):
            return False
        if direction == 'UP':
            return (z == 2) and (state[x][y][z] == Piece.EMPTY or state[x][y][z-1] == Piece.EMPTY or state[x][y][z-2] == Piece.EMPTY)
        elif direction == 'DOWN':
            return (z == 0) and (state[x][y][z] == Piece.EMPTY or state[x][y][z+1] == Piece.EMPTY or state[x][y][z+2] == Piece.EMPTY)
        elif direction == 'LEFT':
            return (x == 2) and (state[x][y][z] == Piece.EMPTY or state[x-1][y][z] == Piece.EMPTY or state[x-2][y][z] == Piece.EMPTY)
        elif direction == 'RIGHT':
            return (x == 0) and (state[x][y][z] == Piece.EMPTY or state[x+1][y][z] == Piece.EMPTY or state[x+2][y][z] == Piece.EMPTY)
        elif direction == 'FRONT':
            return (y == 2) and (state[x][y][z] == Piece.EMPTY or state[x][y-1][z] == Piece.EMPTY or state[x][y-2][z] == Piece.EMPTY)
        elif direction == 'BACK':
            return (y == 0) and (state[x][y][z] == Piece.EMPTY or state[x][y+1][z] == Piece.EMPTY or state[x][y+2][z] == Piece.EMPTY)
        else:
            return False 

    def piece_to_num(self, piece):
        
        if piece == self.player:
            return 1
        elif piece == Piece.EMPTY:
            return 0
        else:
            return -1

    def state2array(self, state):
        state_as_array = []
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    state_as_array.append(self.piece_to_num(state[i][j][k]))
        return np.array(state_as_array)

    def calc_value(self, state):
        x_test = self.state2array(state)
        # print(x_test)
        x_test = tf.reshape(x_test,shape=(1,27))
        
        # print(x_test)
        return self.model.predict(x_test, verbose=0)



# Game Player

In [9]:
class GamePlayer:
    def __init__(self,player1,player2):
      self.player1 = player1
      self.player2 = player2
      self.board = Board()

    def playGame(self):
       self.board = Board()
       while 1 == 1:
          # Player 1 moves
          (x1,y1,z1,dir1) = self.player1.getMove(self.board)
          self.board.move(x1,y1,z1,dir1,self.player1.player)
          # print(self.board.getGameState())
          if self.board.hasWon(self.player1.player):
             return 'player 1 won'

          # Player 2 moves
          (x2,y2,z2,dir2) = self.player2.getMove(self.board)
          self.board.move(x2,y2,z2,dir2,self.player2.player)
          
          # print(self.board.getGameState())
          if self.board.hasWon(self.player2.player):
             return 'player 2 won'

In [10]:
directions = ['UP','DOWN','LEFT','RIGHT','FRONT','BACK']

# Min max agent

In [32]:
import sys


class MinMaxAgent():
    global P1WIN
    P1WIN = 1000000
    global P2WIN
    P2WIN = -1000000

    def __init__(self, player):
        self.player = player
        self.player2 = Board().otherPlayer(player)

    # P1 corner piece -> 1 pt
    # P2 corner piece -> -1 pt
    # P1 runs of 2 -> 10 pts
    # P2 runs of 2 -> -10 pts
    # P1 win in 1's / win in 2's -> 1000000 pts
    # P2 win in 1's / win in 2's -> -1000000 pts
    def evalPos(self, board: Board, p1, p2):
        value = 0
        for x in [0, 2]:
            for y in [0, 2]:
                for z in [0, 2]:
                    if board.pieces[x][y][z] == p1:
                        value += 1
                    if board.pieces[x][y][z] == p2:
                        value -= 1
        for run in board.getWinningRuns():
            if (run[0] == run[1] == p1) or (run[1] == run[2] == p1) or (run[0] == run[2] == p1):
                value += 10
            if (run[0] == run[1] == p2) or (run[1] == run[2] == p2) or (run[0] == run[2] == p2):
                value -= 10
        if board.getWinInOne(p1) != None:
            return P1WIN
        if board.getWinInOne(p2) != None:
            return P2WIN
        return value

    def minMax(self, board, player, depth, alpha, beta):
        maxDepth = 3

        # evaluate the current board using eval(...) and then return its score if this is a leaf node, i.e.,
        # you have reached the maximum depth, or it is a win for one of the players
        cur_score = self.evalPos(board, player, board.otherPlayer(player))
        if depth == maxDepth:
            return (cur_score, None)
        if cur_score in [P2WIN, P1WIN]:
            return (cur_score, None)

        # Otherwise, as in lecture, consider whether this is a maximizing player (O) or minimizing
        # player (X) and perform the min-max algorithm with alpha-beta pruning, plus whatever
        # additional strategies you can come up with

        # Note that when you make a recursive call, you don't need the move, so can just do:
        #         (score,_) = minMax(.....)
        if player == self.player:
            value = P2WIN
            move = board.getRandomMove(player)
            for (x, y, z, dir) in board.getPossibleMoves():
                board_copy = copy.deepcopy(board)
                board_copy.move(x, y, z, dir, self.player)
                new_score = self.minMax(board_copy, self.player2, depth + 1, alpha, beta)[0]
                if new_score > value:
                    print('p1 move updated')
                    value = new_score
                    move = (x, y, z, dir)
                    #alpha = max(alpha, value)
                    #if alpha >= beta:
                    #  break
            # print("returning p1 move")
            return (value, move)
        else:
            value = P1WIN
            move = board.getRandomMove(player)
            for (x, y, z, dir) in board.getPossibleMoves():
                board_copy = copy.deepcopy(board)
                board_copy.move(x, y, z, dir, self.player2)
                new_score = self.minMax(board_copy, self.player, depth + 1, alpha, beta)[0]
                if new_score < value:
                    print('p2 move update')
                    value = new_score
                    move = (x, y, z, dir)
                    #beta = min(beta, value)
                    #if alpha >= beta:
                    #  break
            # print("returning p2 move")
            return (value, move)

    # You will use this function in your interactive version below

    def getMove(self, board):
        (_, move) = self.minMax(board, self.player, 0, -sys.maxsize, sys.maxsize)  # only place we need the move
        if move == None:
            print(board.getGameState())
            print(board.getWinInOne(self.player))
            return board.getWinInOne(self.player) or board.getRandomMove(self.player)
        return move


# Experiments

In [33]:
game = GamePlayer(MinMaxAgent(Piece.RED),MediumAgent(Piece.WHITE))
MediumAgent_wins_second = 0
games_to_play = 1
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  if winner == 'player 2 won':
    MediumAgent_wins_second += 1
print(f'MediumAgent player wins {MediumAgent_wins_second * 100 / games_to_play}% of the time!')

  0%|          | 0/1 [00:00<?, ?it/s]

p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p2 move update
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 move updated
p1 

  0%|          | 0/1 [00:09<?, ?it/s]

p1 move updated
p2 move update
p1 move updated


KeyboardInterrupt: ignored

### DeepAgent vs Easy Agent

In [14]:
import pandas as pd

In [15]:
results = pd.DataFrame(columns=["Player1","Player2", "Games_Won_by_Player1", "Games_Lost_by_Player2"])

In [16]:
from tqdm import tqdm

In [19]:
game = GamePlayer(EasyAgent(Piece.WHITE),DeepAgent(Piece.BLACK))
deep_agent_wins_second = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 2 won':
    # print("deep agent won")
    deep_agent_wins_second += 1
print(f'DeepAgent player wins {deep_agent_wins_second * 100 / games_to_play}% of the time!')
game = GamePlayer(DeepAgent(Piece.WHITE),EasyAgent(Piece.BLACK))
deep_agent_wins_first = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 1 won':
    # print("deep agent won")
    deep_agent_wins_first += 1
print(f'DeepAgent player wins {deep_agent_wins_first * 100 / games_to_play}% of the time!')

 80%|████████  | 16/20 [04:23<01:05, 16.50s/it]


KeyboardInterrupt: ignored

In [ ]:
results.loc[0] = ["DeepAgent", "EasyAgent", deep_agent_wins_first,20-deep_agent_wins_first]

In [ ]:
results.loc[1] = ["EasyAgent", "DeepAgent", 20-deep_agent_wins_second,deep_agent_wins_second]

In [ ]:
results.head()

### DeepAgent vs Medium Agent

In [ ]:
game = GamePlayer(MediumAgent(Piece.WHITE),DeepAgent(Piece.BLACK))
deep_agent_wins_second = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 2 won':
    # print("deep agent won")
    deep_agent_wins_second += 1
print(f'DeepAgent player wins {deep_agent_wins_second * 100 / games_to_play}% of the time!')
game = GamePlayer(DeepAgent(Piece.WHITE),MediumAgent(Piece.BLACK))
deep_agent_wins_first = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 1 won':
    # print("deep agent won")
    deep_agent_wins_first += 1
print(f'DeepAgent player wins {deep_agent_wins_first * 100 / games_to_play}% of the time!')

In [ ]:
results.loc[2] = ["MediumAgent", deep_agent_wins_first,20-deep_agent_wins_first]

In [ ]:
results.loc[3] = ["MediumAgent", 20-deep_agent_wins_second,deep_agent_wins_second]

In [ ]:
results.head()

### DeepAgent vs Hard Agent

In [ ]:
results = pd.DataFrame(columns=["Opponent","Games_Won_Going_First", "Games_Lost_Going_First", "Games_Won_Going_Second","Games_Lost_Going_Second"])

In [ ]:
from tqdm import tqdm

In [13]:
game = GamePlayer(HardAgent(Piece.WHITE),DeepAgent(Piece.BLACK))
deep_agent_wins_second = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 2 won':
    # print("deep agent won")
    deep_agent_wins_second += 1
print(f'DeepAgent player wins {deep_agent_wins_second * 100 / games_to_play}% of the time!')
game = GamePlayer(DeepAgent(Piece.WHITE),HardAgent(Piece.BLACK))
deep_agent_wins_first = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  # print("deep agent didn't win")
  if winner == 'player 1 won':
    # print("deep agent won")
    deep_agent_wins_first += 1
print(f'DeepAgent player wins {deep_agent_wins_first * 100 / games_to_play}% of the time!')

OSError: ignored

In [ ]:
results.loc[4] = ["DeepAgent", "HardAgent", deep_agent_wins_first,20-deep_agent_wins_first]

In [ ]:
results.loc[5] = ["DeepAgent", "HardAgent", 20-deep_agent_wins_first,deep_agent_wins_first]

In [ ]:
results.head()

## Easy vs Medium Agent

In [ ]:
game = GamePlayer(EasyAgent(Piece.WHITE),MediumAgent(Piece.BLACK))
MediumAgent_wins_second = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  if winner == 'player 2 won':
    MediumAgent_wins_second += 1
print(f'MediumAgent player wins {MediumAgent_wins_second * 100 / games_to_play}% of the time!')
game = GamePlayer(MediumAgent(Piece.WHITE),EasyAgent(Piece.BLACK))
MediumAgent_wins_first = 0
games_to_play = 20
for i in tqdm(range(games_to_play)):
  winner = game.playGame()
  if winner == 'player 1 won':
    MediumAgent_wins_first += 1
print(f'MediumAgent player wins {MediumAgent_wins_first * 100 / games_to_play}% of the time!')

In [ ]:
# results.loc[5] = ["EasyAgent", "MediumAgent", MediumAgent_wins_first,20-MediumAgent_wins_first]

In [ ]:
# results.loc[6] = ["MediumAgent", "EasyAgent", 20-MediumAgent_wins_second,MediumAgent_wins_second]

### Notes
The value function determines how good it is to be in state s,
Agent can also learn the value of a state-action pair, which is a q value. The q funciton 
measueres tha value of choosing a particualar action when in a particular state.


Deep Q-Learning replaces the regular Q-table with a neural network. Rather than mapping a state-action pair to a q-value, a neural network maps input states to (action, Q-value) pairs.

The Bellman equation is a recursive equation that relates the value of a state to the values of its successor states. It decomposes the value function into two parts: an immediate reward and the expected discounted value of the next state. The Bellman equation is used to update the value function iteratively, until it converges to the true value function.

https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc

## Good code references:
   * https://github.com/giladariel/TicTacToe_RL/blob/master/DeepTicTacToe_org.py
   * https://github.com/mswang12/minDQN/blob/main/minDQN.py

In [ ]:
### Notes
The value function determines how good it is to be in state s,
Agent can also learn the value of a state-action pair, which is a q value. The q funciton 
measueres tha value of choosing a particualar action when in a particular state.


Deep Q-Learning replaces the regular Q-table with a neural network. Rather than mapping a state-action pair to a q-value, a neural network maps input states to (action, Q-value) pairs.

The Bellman equation is a recursive equation that relates the value of a state to the values of its successor states. It decomposes the value function into two parts: an immediate reward and the expected discounted value of the next state. The Bellman equation is used to update the value function iteratively, until it converges to the true value function.

https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc

## Good code references:
   * https://github.com/giladariel/TicTacToe_RL/blob/master/DeepTicTacToe_org.py
   * https://github.com/mswang12/minDQN/blob/main/minDQN.py